# Task #2

A template code for training an RBM on Rydberg atom data (the full dataset) is provided below. For the first part of this task (determining the minimum number of hidden units), start with 20 hidden units. 

Imports and loadining in data:

In [1]:
from pathlib import Path
import numpy as np
import torch
from RBM_helper import RBM
import csv
import Rydberg_energy_calculator
from datetime import datetime

training_data = torch.from_numpy(np.loadtxt("Rydberg_data.txt"))
n_vis = training_data.shape[1]

target = 0.0001
exact_energy = -4.1203519096
print("Exact energy: ",exact_energy)
output_dir = Path("training_logs")

if not output_dir.exists():
    output_dir.mkdir(parents=True)
differences = {}
training_num = training_data.shape[0]
print(training_num)

def train(n_hin):
    rbm = RBM(n_vis, n_hin)
    epochs = 1000
    num_samples = 2000
    print('Starting training on n_h = ', n_hin)
    differences = []
    for e in range(1, epochs+1):
        # do one epoch of training
        lr = .1*np.exp(-e/epochs)+1e-4
        rbm.train(training_data, lr=lr)

        # now generate samples and calculate the energy
        if e % 100 == 0:
            print("\nEpoch: ", e)
            # print("Sampling...")

            init_state = torch.zeros(num_samples, n_vis)
            # idx = np.random.rand(training_num)
            RBM_samples = rbm.draw_samples(100, init_state)

            # print("Done sampling. Calculating energy...")

            energies = Rydberg_energy_calculator.energy(RBM_samples, rbm.wavefunction)
            energy_difference = abs(exact_energy - energies.item())
            differences.append(energy_difference)
            print("Energy difference: ", energy_difference)
    return differences

Exact energy:  -4.1203519096
20000


In [ ]:
from multiprocessing import Pool

with Pool(16) as p:
    results = p.map(train, range(20, 500, 20))

Starting training on n_h = Starting training on n_h = Starting training on n_h = Starting training on n_h = Starting training on n_h = Starting training on n_h = Starting training on n_h =  Starting training on n_h =  Starting training on n_h =     2060   4080
120
100160180140







Epoch:  100
Energy difference:  0.0007025816205450397

Epoch:  100
Energy difference:  0.0014494232242681093

Epoch:  100
Energy difference:  0.0015504857419443852

Epoch:  100
Energy difference:  0.0013437700676854902

Epoch:  100
Energy difference:  0.00152831364534034

Epoch:  100
Energy difference:  0.0010909322492738127

Epoch:  100
Energy difference:  0.0017779533968456107

Epoch:  100
Energy difference:  0.0015779668493811627

Epoch:  100
Energy difference:  0.0011502170167068115

Epoch:  200
Energy difference:  0.0006641824233124538

Epoch:  200
Energy difference:  0.0011177536903961283

Epoch:  200
Energy difference:  0.0009909075029286285

Epoch:  200
Energy difference:  0.0013755140318680859

Ep

../aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
../aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
../aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
../aten/src/ATen/native/BinaryOps.cpp:81: UserWarning: Integer division of tensors using div or / is deprecated, and in a future release div will perform true division as in Python 3. Use true_divide or floor_divide (// in Python) instead.
../aten/src/ATen/native/BinaryOps.cpp:81

In [41]:
print(results)

[[0.0006310583671433889, 0.000921435257033032, 0.0008249763915291197, 0.0002358291517472466, 0.0006430586721304365, 0.0003901989711065923, 0.0007728823973174315, 0.0006230158105875461, 0.0005175208083594285, 0.0004245495499004548], [0.0008475099397609043, 0.000907373093994579, 0.0011729905574560107, 0.0003560218662768122, 0.0008484090512315134, 0.0007706714996400521, 0.00020168960690103432, 0.0005729445511457598, 0.0002453111560356547, 0.0004575601778320504], [0.0014029966640736191, 0.0006042227534681643, 0.0007287060625564834, 0.0009619315357349834, 0.0009193632576200983, 0.0005213437419566702, 0.0010433943246521693, 0.00065047332706758, 0.0003492967387073165, 0.0006075443657289625], [0.0012212809912872657, 0.0012316421801124022, 0.0008570149984956643, 0.0004220908718819416, 0.001006561875807499, 0.000722634701970648, 0.0006617482548394449, 0.0005886573291196129, 0.0007469560301824885, 0.0002095538275765918], [0.0012133892398695778, 0.0009662753229386922, 0.0010948102160490691, 0.0010

In [42]:
with open(str(output_dir / f"task2.csv"), 'w') as f:
    w = csv.writer(f)
    w.writerows(results)